# Scrabu Prototype (for a List of Shipment Numbers)

This notebook is for creating a prototype of the Scrabu project. The goal is to download DHL pages for a specific shipment number and scrap the shipment information from it.

#### Configure Logger

In [10]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.basicConfig(format='%(asctime)s %(levelname)s: %(message)s', level=logging.INFO, datefmt='%I:%M:%S')

#### Generate a list of shipment numbers with the check digit calculation

In [11]:
def generate_shipment_numbers(shipment_number=None, size=2): #340434188193324407|340434188193323500
    import numpy as np
    from functools import reduce
    logger.info("Generating shipment numbers with seed: {}".format(shipment_number))
    multiplier = [3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3]
    shipment_numbers_list = []
    for i in range(0, size):
        shipment_number = (shipment_number // 10) + 1
        shipment_number_l = list(map(int, str(shipment_number)))
        multiply_number = np.multiply(multiplier, shipment_number_l)
        sum = reduce(lambda x, y: x+y, multiply_number)
        pz = (10 - sum % 10)
        shipment_number_l.append(0 if pz==10 else pz)
        shipment_number = reduce(lambda x,y: x * 10 + y, shipment_number_l)
        shipment_number_str = str(shipment_number).rjust(20, '0')
        shipment_numbers_list.append(shipment_number_str)
    logger.info("Generated {} unique shipment numbers".format(len(set(shipment_numbers_list))))
    return shipment_numbers_list

#### Download the HTML content for a list of shipment numbers

In [12]:
def request(shipment_number=None, start_url="https://www.dhl.de/int-verfolgen/search?language=de&lang=de&domain=de&piececode="):
    import requests
    import numpy as np
    logger.debug("Making HTTP request for shipment number {}".format(shipment_number))
    request_url = start_url + str(shipment_number)
    return requests.get(request_url).content

#### Parsing HTML and converting it into JSON

In [13]:
def html_to_json(html):
    from lxml import etree
    import json
    logger.debug("Converting HTML to JSON")
    
    html_tree = etree.HTML(html)
    
    def clean_json(dirty_json):
        start = dirty_json.find('JSON.parse(')
        end = dirty_json.find('"),', start)
        cjson = dirty_json[start:end]
        cjson = cjson.replace('JSON.parse("', '')
        cjson = cjson.replace('\\', '')
        return cjson
    
    def find_json_element(html_tree):
        json_element = html_tree.xpath('//div')
        return str(etree.tostring(json_element[0]))
    
    dirty_json = find_json_element(html_tree)
    json_string = clean_json(dirty_json)
    return json.loads(json_string)

#### Structure the shipment details in JSON format

In [14]:
def shipment_details(shipment_details_json, start_url="https://www.dhl.de/int-verfolgen/search?language=de&lang=de&domain=de&piececode=", shipment_number="00340434188193323500"):
    import datetime
    logger.debug("Preparing JSON for persistance")
    delivery_history_dict = {}
    delivery_history_dict["shipment_number"] = shipment_details_json["sendungen"][0]["sendungsdetails"]["sendungsnummern"].get("sendungsnummer")
    delivery_history_dict["crawltime"] = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    delivery_history_dict["url"] = start_url + str(shipment_number) 
    delivery_history_dict["events"] = shipment_details_json["sendungen"][0]["sendungsdetails"]["sendungsverlauf"].get("events", [])
    return delivery_history_dict

In [15]:
def save_dictionary(shipment_history=None, filename=None):
    import json
    logger.info("Writing file {}".format(filename))
    with open(filename, 'w') as f:
        json.dump(shipment_history, f)

#### This function summarizes all previous steps for a single shipment number

In [16]:
def process_shipment_number(shipment_number):
    import time
    time.sleep(1.0)
    html_response = request(shipment_number=shipment_number)
    shipment_details_json = html_to_json(html_response)
    shipment_history = shipment_details(shipment_details_json, shipment_number=shipment_number)
    if len(shipment_history['events']) > 0:
        save_dictionary(shipment_history, filename="../data/{}.json".format(shipment_number))
    else:
        logger.info("No events found for shipment number {}".format(shipment_number))

#### Multi-threading

In [20]:
def main(shipment_number=None, size=None, max_workers=None, start_url="https://www.dhl.de/int-verfolgen/search?language=de&lang=de&domain=de&piececode="):
    from concurrent import futures
    import concurrent
    from concurrent.futures import ThreadPoolExecutor as PoolExecutor
    
    shipment_numbers = generate_shipment_numbers(shipment_number=shipment_number, size=size)
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_url = {executor.submit(process_shipment_number, sn): sn for sn in shipment_numbers}
        for future in concurrent.futures.as_completed(future_to_url):
            url = future_to_url[future]
            try:
                data = future.result()
            except Exception as exc:
                print('%r generated an exception: %s' % (url, exc))

09:50:14 INFO: No events found for shipment number 00340434188192332786


In [21]:
main(shipment_number=340434188193324407, size=10000, max_workers=4)

09:50:15 INFO: Generating shipment numbers with seed: 340434188193324407
09:50:15 INFO: No events found for shipment number 00340434188193229528
09:50:15 INFO: No events found for shipment number 00340434188192332793
09:50:15 INFO: No events found for shipment number 00340434188192332809
09:50:15 INFO: Generated 10000 unique shipment numbers
09:50:15 INFO: No events found for shipment number 00340434188192332816
09:50:15 INFO: No events found for shipment number 00340434188193229535
09:50:15 INFO: No events found for shipment number 00340434188193229542
09:50:15 INFO: No events found for shipment number 00340434188193229559
09:50:16 INFO: No events found for shipment number 00340434188192332823
09:50:16 INFO: No events found for shipment number 00340434188192332830
09:50:16 INFO: No events found for shipment number 00340434188193229566
09:50:16 INFO: No events found for shipment number 00340434188192332847
09:50:16 INFO: No events found for shipment number 00340434188192332854
09:50:17

09:50:28 INFO: No events found for shipment number 00340434188192333233
09:50:28 INFO: Writing file ../data/00340434188193324766.json
09:50:28 INFO: No events found for shipment number 00340434188193229900
09:50:28 INFO: No events found for shipment number 00340434188193229917
09:50:28 INFO: No events found for shipment number 00340434188192333240
09:50:28 INFO: No events found for shipment number 00340434188193229924
09:50:28 INFO: Writing file ../data/00340434188193324773.json
09:50:28 INFO: No events found for shipment number 00340434188192333257
09:50:29 INFO: Writing file ../data/00340434188193324780.json
09:50:29 INFO: No events found for shipment number 00340434188193229931
09:50:29 INFO: No events found for shipment number 00340434188192333264
09:50:29 INFO: Writing file ../data/00340434188193324797.json
09:50:29 INFO: No events found for shipment number 00340434188192333271
09:50:29 INFO: Writing file ../data/00340434188193324803.json
09:50:29 INFO: No events found for shipmen

09:50:40 INFO: Writing file ../data/00340434188193325145.json
09:50:40 INFO: No events found for shipment number 00340434188192333660
09:50:40 INFO: No events found for shipment number 00340434188192333677
09:50:41 INFO: Writing file ../data/00340434188193325152.json
09:50:41 INFO: No events found for shipment number 00340434188193230296
09:50:41 INFO: No events found for shipment number 00340434188192333684
09:50:41 INFO: Writing file ../data/00340434188193325169.json
09:50:41 INFO: No events found for shipment number 00340434188192333691
09:50:41 INFO: No events found for shipment number 00340434188193230302
09:50:41 INFO: Writing file ../data/00340434188193325176.json
09:50:41 INFO: No events found for shipment number 00340434188193230319
09:50:41 INFO: No events found for shipment number 00340434188193230326
09:50:42 INFO: No events found for shipment number 00340434188192333707
09:50:42 INFO: No events found for shipment number 00340434188192333714
09:50:42 INFO: Writing file ../d

09:50:53 INFO: Writing file ../data/00340434188193325534.json
09:50:53 INFO: No events found for shipment number 00340434188192334100
09:50:53 INFO: Writing file ../data/00340434188193325541.json
09:50:53 INFO: No events found for shipment number 00340434188192334117
09:50:53 INFO: No events found for shipment number 00340434188193230661
09:50:53 INFO: No events found for shipment number 00340434188192334124
09:50:53 INFO: Writing file ../data/00340434188193325558.json
09:50:54 INFO: No events found for shipment number 00340434188193230678
09:50:54 INFO: No events found for shipment number 00340434188193230685
09:50:54 INFO: No events found for shipment number 00340434188192334131
09:50:54 INFO: Writing file ../data/00340434188193325565.json
09:50:54 INFO: No events found for shipment number 00340434188193230692
09:50:54 INFO: Writing file ../data/00340434188193325572.json
09:50:54 INFO: No events found for shipment number 00340434188192334148
09:50:54 INFO: No events found for shipmen

09:51:05 INFO: No events found for shipment number 00340434188193231033
09:51:05 INFO: Writing file ../data/00340434188193325923.json
09:51:05 INFO: No events found for shipment number 00340434188193231040
09:51:05 INFO: Writing file ../data/00340434188193325930.json
09:51:05 INFO: No events found for shipment number 00340434188193231057
09:51:06 INFO: No events found for shipment number 00340434188192334544
09:51:06 INFO: No events found for shipment number 00340434188192334551
09:51:06 INFO: Writing file ../data/00340434188193325947.json
09:51:06 INFO: No events found for shipment number 00340434188192334568
09:51:06 INFO: No events found for shipment number 00340434188192334575
09:51:06 INFO: Writing file ../data/00340434188193325954.json
09:51:06 INFO: No events found for shipment number 00340434188193231064
09:51:07 INFO: No events found for shipment number 00340434188193231071
09:51:07 INFO: Writing file ../data/00340434188193325961.json
09:51:07 INFO: Writing file ../data/003404

...

10:39:05 INFO: No events found for shipment number 00340434188193423032
10:39:05 INFO: No events found for shipment number 00340434188193423049
10:39:06 INFO: Writing file ../data/00340434188193318833.json
10:39:06 INFO: Writing file ../data/00340434188193318840.json
10:39:06 INFO: Writing file ../data/00340434188193318864.json
10:39:06 INFO: Writing file ../data/00340434188193318857.json
10:39:06 INFO: No events found for shipment number 00340434188193423056
10:39:06 INFO: No events found for shipment number 00340434188193423063
10:39:06 INFO: No events found for shipment number 00340434188193423070
10:39:07 INFO: No events found for shipment number 00340434188193423087
10:39:07 INFO: Writing file ../data/00340434188193318871.json
10:39:07 INFO: Writing file ../data/00340434188193318888.json
10:39:07 INFO: No events found for shipment number 00340434188193423094
10:39:07 INFO: No events found for shipment number 00340434188193423100
10:39:07 INFO: Writing file ../data/0034043418819331

10:39:24 INFO: Writing file ../data/00340434188193319397.json
10:39:24 INFO: Writing file ../data/00340434188193319403.json
10:39:24 INFO: Writing file ../data/00340434188193319427.json
10:39:24 INFO: Writing file ../data/00340434188193319410.json
10:39:24 INFO: No events found for shipment number 00340434188193423698
10:39:24 INFO: No events found for shipment number 00340434188193423704
10:39:25 INFO: No events found for shipment number 00340434188193423711
10:39:25 INFO: No events found for shipment number 00340434188193423728
10:39:25 INFO: Writing file ../data/00340434188193319434.json
10:39:25 INFO: Writing file ../data/00340434188193319441.json
10:39:26 INFO: Writing file ../data/00340434188193319458.json
10:39:26 INFO: Writing file ../data/00340434188193319465.json
10:39:26 INFO: No events found for shipment number 00340434188193423735
10:39:26 INFO: No events found for shipment number 00340434188193423742
10:39:26 INFO: No events found for shipment number 00340434188193423759


10:39:43 INFO: No events found for shipment number 00340434188193424336
10:39:43 INFO: Writing file ../data/00340434188193319977.json
10:39:43 INFO: Writing file ../data/00340434188193319984.json
10:39:43 INFO: No events found for shipment number 00340434188193424343
10:39:43 INFO: Writing file ../data/00340434188193319991.json
10:39:43 INFO: No events found for shipment number 00340434188193424350
10:39:43 INFO: No events found for shipment number 00340434188193424367
10:39:44 INFO: No events found for shipment number 00340434188193424374
10:39:44 INFO: Writing file ../data/00340434188193320003.json
10:39:44 INFO: Writing file ../data/00340434188193320010.json
10:39:44 INFO: No events found for shipment number 00340434188193424381
10:39:44 INFO: Writing file ../data/00340434188193320027.json
10:39:45 INFO: No events found for shipment number 00340434188193424398
10:39:45 INFO: Writing file ../data/00340434188193320034.json
10:39:45 INFO: No events found for shipment number 00340434188

10:40:24 INFO: Writing file ../data/00340434188193321215.json
10:40:24 INFO: Writing file ../data/00340434188193321222.json
10:40:24 INFO: Writing file ../data/00340434188193321208.json
10:40:24 INFO: Writing file ../data/00340434188193321239.json
10:40:25 INFO: Writing file ../data/00340434188193321246.json
10:40:25 INFO: Writing file ../data/00340434188193321260.json
10:40:25 INFO: Writing file ../data/00340434188193321253.json
10:40:25 INFO: Writing file ../data/00340434188193321277.json
10:40:26 INFO: Writing file ../data/00340434188193321284.json
10:40:26 INFO: Writing file ../data/00340434188193321307.json
10:40:27 INFO: Writing file ../data/00340434188193321314.json
10:40:27 INFO: Writing file ../data/00340434188193321291.json
10:40:28 INFO: Writing file ../data/00340434188193321321.json
10:40:28 INFO: Writing file ../data/00340434188193321338.json
10:40:28 INFO: Writing file ../data/00340434188193321352.json
10:40:28 INFO: Writing file ../data/00340434188193321345.json
10:40:29

10:41:06 INFO: Writing file ../data/00340434188193322533.json
10:41:07 INFO: Writing file ../data/00340434188193322540.json
10:41:07 INFO: Writing file ../data/00340434188193322557.json
10:41:07 INFO: Writing file ../data/00340434188193322564.json
10:41:08 INFO: Writing file ../data/00340434188193322571.json
10:41:08 INFO: Writing file ../data/00340434188193322588.json
10:41:08 INFO: Writing file ../data/00340434188193322595.json
10:41:09 INFO: Writing file ../data/00340434188193322601.json
10:41:09 INFO: Writing file ../data/00340434188193322618.json
10:41:09 INFO: Writing file ../data/00340434188193322625.json
10:41:09 INFO: Writing file ../data/00340434188193322632.json
10:41:10 INFO: Writing file ../data/00340434188193322649.json
10:41:10 INFO: Writing file ../data/00340434188193322656.json
10:41:10 INFO: Writing file ../data/00340434188193322663.json
10:41:11 INFO: Writing file ../data/00340434188193322670.json
10:41:11 INFO: Writing file ../data/00340434188193322687.json
10:41:11